In [39]:
# Report mentioning who did what, and how the project evolved with time, report should be small.
# Prepared code and apply on new data.
# Sensitivity Vs. 1-Specificity with area as threshold, PR curve
# ! pip install openslide
import openslide
import numpy as np
import matplotlib.pyplot as plt

# def extract_patches(slide_path, window_size=256, stride=0):
#     """
#     Extract patches and their coordinates from an SVS image.

#     Parameters:
#     - slide_path: Path to the .svs file.
#     - window_size: The size of each window (height and width in pixels).
#     - stride: The step size between windows in pixels.

#     Returns:
#     - A list of tuples, each containing a patch and its (x, y) coordinates.
#     """
#     slide = openslide.OpenSlide(slide_path)
#     width, height = slide.dimensions
#     patches = []
    
#     for y in range(0, height - window_size + 1, stride):
#         for x in range(0, width - window_size + 1, stride):
#             region = slide.read_region((x, y), 0, (window_size, window_size))
#             region_array = np.array(region)[:, :, :3]
#             # print(region_array.shape, 'region array')
#             patches.append((region_array, (x, y)))
#     print(x, y)
#     plt.imshow(region_array)
#     plt.show()
#     slide.close()
#     return patches

# # def stitch_patches(patches, dimensions):
# #     """
# #     Reassemble the image from patches.

# #     Parameters:
# #     - patches: A list of tuples, each containing a patch and its (x, y) coordinates.
# #     - dimensions: The dimensions (width, height) of the full image.

# #     Returns:
# #     - An image reconstructed from the patches.
# #     """
# #     # Create an empty array to hold the full image
# #     full_image = np.zeros((dimensions[1], dimensions[0], 3), dtype=np.uint8)
    
# #     for patch, (x, y) in patches:
# #         full_image[y:y+patch.shape[0], x:x+patch.shape[1], :] = patch
        
# #     return full_image

# # Path to your SVS file
# slide_path = 'P44F_PAS.svs'

# # Extract patches and their coordinates
# patches = extract_patches(slide_path)

# # Stitch the patches back into the full image
# slide = openslide.OpenSlide(slide_path)
# original_dimensions = slide.dimensions
# slide.close()

# reconstructed_image = stitch_patches(patches, original_dimensions)
# print("Image reassembled successfully.")

In [17]:
# def norm_HnE(img, Io=240, alpha=1, beta=0.15):


#     ######## Step 1: Convert RGB to OD ###################
#     ## reference H&E OD matrix.
#     #Can be updated if you know the best values for your image. 
#     #Otherwise use the following default values. 
#     #Read the above referenced papers on this topic. 
#     HERef = np.array([[0.5626, 0.2159],
#                       [0.7201, 0.8012],
#                       [0.4062, 0.5581]])
#     ### reference maximum stain concentrations for H&E
#     maxCRef = np.array([1.9705, 1.0308])
    
    
#     # extract the height, width and num of channels of image
#     h, w, c = img.shape
    
#     # reshape image to multiple rows and 3 columns.
#     #Num of rows depends on the image size (wxh)
#     img = img.reshape((-1,3))
    
#     # calculate optical density
#     # OD = −log10(I)  
#     #OD = -np.log10(img+0.004)  #Use this when reading images with skimage
#     #Adding 0.004 just to avoid log of zero. 
    
#     OD = -np.log10((img.astype(np.float)+1)/Io) #Use this for opencv imread
#     #Add 1 in case any pixels in the image have a value of 0 (log 0 is indeterminate)
    
    
#     ############ Step 2: Remove data with OD intensity less than β ############
#     # remove transparent pixels (clear region with no tissue)
#     ODhat = OD[~np.any(OD < beta, axis=1)] #Returns an array where OD values are above beta
#     #Check by printing ODhat.min()
    
#     ############# Step 3: Calculate SVD on the OD tuples ######################
#     #Estimate covariance matrix of ODhat (transposed)
#     # and then compute eigen values & eigenvectors.
#     eigvals, eigvecs = np.linalg.eigh(np.cov(ODhat.T))
    
    
#     ######## Step 4: Create plane from the SVD directions with two largest values ######
#     #project on the plane spanned by the eigenvectors corresponding to the two 
#     # largest eigenvalues    
#     That = ODhat.dot(eigvecs[:,1:3]) #Dot product
    
#     ############### Step 5: Project data onto the plane, and normalize to unit length ###########
#     ############## Step 6: Calculate angle of each point wrt the first SVD direction ########
#     #find the min and max vectors and project back to OD space
#     phi = np.arctan2(That[:,1],That[:,0])
    
#     minPhi = np.percentile(phi, alpha)
#     maxPhi = np.percentile(phi, 100-alpha)
    
#     vMin = eigvecs[:,1:3].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
#     vMax = eigvecs[:,1:3].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)
    
    
#     # a heuristic to make the vector corresponding to hematoxylin first and the 
#     # one corresponding to eosin second
#     if vMin[0] > vMax[0]:    
#         HE = np.array((vMin[:,0], vMax[:,0])).T
        
#     else:
#         HE = np.array((vMax[:,0], vMin[:,0])).T
    
    
#     # rows correspond to channels (RGB), columns to OD values
#     Y = np.reshape(OD, (-1, 3)).T
    
#     # determine concentrations of the individual stains
#     C = np.linalg.lstsq(HE,Y, rcond=None)[0]
    
#     # normalize stain concentrations
#     maxC = np.array([np.percentile(C[0,:], 99), np.percentile(C[1,:],99)])
#     tmp = np.divide(maxC,maxCRef)
#     C2 = np.divide(C,tmp[:, np.newaxis])
    
#     ###### Step 8: Convert extreme values back to OD space
#     # recreate the normalized image using reference mixing matrix 
    
#     Inorm = np.multiply(Io, np.exp(-HERef.dot(C2)))
#     Inorm[Inorm>255] = 254
#     Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)  
    
#     # Separating H and E components
    
#     H = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,0], axis=1).dot(np.expand_dims(C2[0,:], axis=0))))
#     H[H>255] = 254
#     H = np.reshape(H.T, (h, w, 3)).astype(np.uint8)
    
#     E = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,1], axis=1).dot(np.expand_dims(C2[1,:], axis=0))))
#     E[E>255] = 254
#     E = np.reshape(E.T, (h, w, 3)).astype(np.uint8)
    
#     return (Inorm, H, E)


In [33]:
import openslide
import tifffile as tiff
slide_path = 'P44F_PAS.svs'
slide = openslide.OpenSlide(slide_path)
from openslide.deepzoom import DeepZoomGenerator
tiles = DeepZoomGenerator(slide, tile_size=512, overlap=0, limit_bounds=False)
print("The number of levels in the tiles object are: ", tiles.level_count)
print("The dimensions of data in each level are: ", tiles.level_dimensions)
#Total number of tiles in the tiles object
print("Total number of tiles = : ", tiles.tile_count)

The number of levels in the tiles object are:  16
The dimensions of data in each level are:  ((1, 1), (2, 2), (3, 3), (6, 6), (11, 11), (21, 22), (41, 43), (82, 85), (164, 169), (327, 338), (654, 676), (1308, 1351), (2616, 2701), (5231, 5402), (10461, 10804), (20922, 21608))
Total number of tiles = :  2405


In [34]:
import os
###### Saving each tile to local directory
orig_tile_dir_name = 'Patches'
cols, rows = tiles.level_tiles[15]
i=1
for row in range(rows):
    for col in range(cols):
        tile_name =   'patch_' + str(i)      #str(col) + "_" + str(row)
        #tile_name = os.path.join(tile_dir, '%d_%d' % (col, row))
        #print("Now processing tile with title: ", tile_name)
        temp_tile = tiles.get_tile(15, (col, row))
        temp_tile_RGB = temp_tile.convert('RGB')
        temp_tile_np = np.array(temp_tile_RGB)
        print(temp_tile_np.mean(), temp_tile_np.std(), os.path.join(orig_tile_dir_name, tile_name) + "_original.tif")
        #Save original tile
        if temp_tile_np.mean() < 230 and temp_tile_np.std() > 15:
            i+=1
            tiff.imwrite(os.path.join(orig_tile_dir_name, tile_name) + "_original.tif", temp_tile_np)
#         if temp_tile_np.mean() < 230 and temp_tile_np.std() > 15:
#             print("Processing tile number:", tile_name)
#             norm_img, H_img, E_img = norm_HnE(temp_tile_np, Io=240, alpha=1, beta=0.15)
#         #Save the norm tile, H and E tiles
            
#             tiff.imsave(norm_tile_dir_name+tile_name + "_norm.tif", norm_img)
#             tiff.imsave(H_tile_dir_name+tile_name + "_H.tif", H_img)
#             tiff.imsave(E_tile_dir_name+tile_name + "_E.tif", E_img)
            
        else:
            # print("NOT PROCESSING TILE:", tile_name)
            pass


243.3757527669271 2.122565443189113 Patches/patch_1_original.tif
243.2342987060547 2.345730532670152 Patches/patch_1_original.tif
243.19889322916666 2.0470600901906346 Patches/patch_1_original.tif
243.11812591552734 2.1536065713151546 Patches/patch_1_original.tif
243.13024393717447 2.03175116951944 Patches/patch_1_original.tif
243.08180363972983 2.068395439088079 Patches/patch_1_original.tif
243.00237147013345 2.184551635995789 Patches/patch_1_original.tif
242.86709340413412 2.3137979626023504 Patches/patch_1_original.tif
242.5247891743978 2.258595398165401 Patches/patch_1_original.tif
242.27063751220703 6.202140985336668 Patches/patch_1_original.tif
242.24987920125326 2.3758517932361882 Patches/patch_1_original.tif
242.19778442382812 2.316781095495404 Patches/patch_1_original.tif
241.96649424235025 2.8724856504355643 Patches/patch_1_original.tif
241.9257074991862 2.77786503490628 Patches/patch_1_original.tif
242.01078669230142 2.4630115620526434 Patches/patch_1_original.tif
241.875170

In [ ]:
# import os
# tile_dir = "images/saved_tiles/original_tiles/"
# for row in range(rows):
#     for col in range(cols):
#         tile_name = os.path.join(tile_dir, '%d_%d' % (col, row))
#         print("Now saving tile with title: ", tile_name)
#         temp_tile = tiles.get_tile(16, (col, row))
#         temp_tile_RGB = temp_tile.convert('RGB')
#         temp_tile_np = np.array(temp_tile_RGB)
#         plt.imsave(tile_name + ".png", temp_tile_np)


In [7]:
# print(len(patches))
# import matplotlib.pyplot as plt
# # plt.imshow(patches[0])

27054


In [11]:
# import openslide
# import numpy as np
# import matplotlib.pyplot as plt

# def extract_patches(slide_path, window_size=512, stride=256):
#     slide = openslide.OpenSlide(slide_path)
#     width, height = slide.dimensions

#     # Calculate the number of patches that can be extracted in both dimensions
#     num_patches_x = (width - window_size) // stride + 1
#     num_patches_y = (height - window_size) // stride + 1

#     # Initialize the array to store patches and coordinates
#     patches = np.zeros((num_patches_y * num_patches_x, window_size, window_size, 3), dtype=np.uint8)
#     coordinates = np.zeros((num_patches_y * num_patches_x, 2), dtype=np.int32)

#     # Patch extraction
#     patch_index = 0
#     for y in range(0, height - window_size + 1, stride):
#         for x in range(0, width - window_size + 1, stride):
#             region = slide.read_region((x, y), 0, (window_size, window_size))
#             region_array = np.array(region)[:, :, :3]  # Drop alpha channel
#             patches[patch_index] = region_array
#             coordinates[patch_index] = [x, y]
#             patch_index += 1

#     slide.close()
#     return patches, coordinates

# def stitch_patches(patches, coordinates, dimensions):
#     full_image = np.zeros((dimensions[1], dimensions[0], 3), dtype=np.uint8)
#     for i, (x, y) in enumerate(coordinates):
#         full_image[y:y+patches.shape[1], x:x+patches.shape[2], :] = patches[i]
#     return full_image

# # Path to your SVS file
# slide_path = 'P44F_PAS.svs'

# # Extract patches and their coordinates
# patches, coordinates = extract_patches(slide_path)

# # Stitch the patches back into the full image
# slide = openslide.OpenSlide(slide_path)
# original_dimensions = slide.dimensions
# slide.close()

# reconstructed_image = stitch_patches(patches, coordinates, original_dimensions)
# print("Image reassembled successfully.")


Image reassembled successfully.


In [12]:
# patches.shape

(6640, 512, 512, 3)

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from skimage import io, color, filters, img_as_float
# from skimage.filters import gabor_kernel
# from scipy.ndimage import convolve
# from sklearn.decomposition import PCA
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler
# import cv2
# import time

# start_time = time.time()

# # Read and convert the image
# image = io.imread('P44F_PAS_annotation.tif')
# image_gray = color.rgb2gray(image)
# plt.figure()
# plt.imshow(image_gray, cmap='gray')
# plt.axis('on')

# # Image dimensions
# numRows, numCols = image_gray.shape

# # Gabor filter parameters
# wavelengthMin = 4 / np.sqrt(2)
# wavelengthMax = np.hypot(numRows, numCols)
# n = int(np.floor(np.log2(wavelengthMax / wavelengthMin)))
# wavelengths = 2 ** np.arange(0, n-2) * wavelengthMin

# deltaTheta = 30
# orientations = np.arange(0, 180, deltaTheta)

# # Create Gabor filters
# gabors = [gabor_kernel(frequency=1.0/l, theta=np.deg2rad(theta)) for l in wavelengths for theta in orientations]
# gabormag = np.stack([convolve(image_gray, np.real(g), mode='wrap') for g in gabors], axis=2)

# # Apply Gaussian smoothing
# for i in range(len(gabors)):
#     sigma = 0.5 * wavelengths[i // len(orientations)]
#     K = 3
#     gabormag[:, :, i] = filters.gaussian(gabormag[:, :, i], sigma=K * sigma)

# # Feature matrix construction
# X, Y = np.meshgrid(np.arange(numCols), np.arange(numRows))
# featureSet = np.dstack([gabormag, X, Y])

# # Reshape and standardize features
# numPoints = numRows * numCols
# X = featureSet.reshape((numPoints, -1))
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# # PCA for dimensionality reduction
# pca = PCA(n_components=1)
# X_pca = pca.fit_transform(X)
# feature2DImage = X_pca.reshape((numRows, numCols))
# print('here1')
# plt.figure()
# plt.imshow(feature2DImage, cmap='jet')
# plt.axis('on')
# plt.show()
# # K-means clustering
# kmeans = KMeans(n_clusters=2, n_init=5)
# L = kmeans.fit_predict(X)
# L = L.reshape((numRows, numCols))
# print('here2', L.shape, "L", np.unique(L))
# plt.figure()
# plt.imshow(color.label2rgb(L, image_gray, kind='avg'))
# plt.show()

# # Segment the image
# Aseg1 = np.zeros_like(image)
# Aseg2 = np.zeros_like(image)
# BW = L == 2
# BW = np.repeat(BW[:, :, np.newaxis], 3, axis=2)
# Aseg1[BW] = image[BW]
# Aseg2[~BW] = image[~BW]
# print('here3')
# plt.figure()
# plt.imshow(np.hstack([Aseg1, Aseg2]))
# plt.show()

# print("Elapsed time: {:.2f} seconds".format(time.time() - start_time))
# plt.show()


# inverted_L = 1 - L
# # print(inverted_L)
# L = np.repeat(inverted_L[:, :, np.newaxis], 3, axis=2)
# final = L*image
# plt.imshow(final)
# plt.show()

In [10]:
# import openslide

# # Path to the SVS file
# file_path = 'P44F_PAS.svs'

# # Open the slide using OpenSlide
# slide = openslide.OpenSlide(file_path)

# # Get dimensions of the slide
# width, height = slide.dimensions

# # Print the dimensions
# print("Width:", width, "Height:", height)


Width: 20922 Height: 21608


In [1]:
# import openslide
# import numpy as np
# import matplotlib.pyplot as plt
# import os
# from PIL import Image

# def extract_patches(slide_path, save_dir, window_size=256, stride=128, background_threshold=200):
#     # Ensure the directory exists
#     if not os.path.exists(save_dir):
#         os.makedirs(save_dir)

#     slide = openslide.OpenSlide(slide_path)
#     width, height = slide.dimensions

#     # Calculate the number of patches
#     num_patches_x = (width - window_size) // stride + 1
#     num_patches_y = (height - window_size) // stride + 1

#     # Patch extraction
#     patch_index = 0
#     for y in range(0, height - window_size + 1, stride):
#         for x in range(0, width - window_size + 1, stride):
#             region = slide.read_region((x, y), 0, (window_size, window_size))
#             region_array = np.array(region)[:, :, :3]  # Drop alpha channel
            
#             # Calculate the average intensity of the patch to determine if it is mostly background
#             if np.mean(region_array) < background_threshold:
#                 img = Image.fromarray(region_array)
#                 img.save(os.path.join(save_dir, f'patch_{patch_index}.png'))
#             patch_index += 1

#     slide.close()

# # Path to your SVS file
# slide_path = 'P44F_PAS.svs'
# save_directory = 'patches_directory'

# # Extract patches and save them in the specified directory
# extract_patches(slide_path, save_directory)

# print("Patches without excessive background have been saved successfully.")


Patches without excessive background have been saved successfully.


In [ ]:
import skimage as sk
for i in range(len(os.listdir(patches_directory))):
    img = sk.io.imread(os.listdir(patches_directory)[i])
    
    

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from skimage import io, color, filters, img_as_float
# from skimage.filters import gabor_kernel
# from scipy.ndimage import convolve
# from sklearn.decomposition import PCA
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler
# import cv2
# import time

# start_time = time.time()
# for i in range(len(os.listdir('patches_directory'))):
#     # Read and convert the image
#     print(os.listdir('patches_directory')[i])
#     image = io.imread(os.path.join('patches_directory', os.listdir('patches_directory')[i]))
#     image_gray = color.rgb2gray(image)
#     plt.figure()
#     plt.imshow(image_gray, cmap='gray')
#     plt.axis('on')

#     # Image dimensions
#     numRows, numCols = image_gray.shape

#     # Gabor filter parameters
#     wavelengthMin = 4 / np.sqrt(2)
#     wavelengthMax = np.hypot(numRows, numCols)
#     n = int(np.floor(np.log2(wavelengthMax / wavelengthMin)))
#     wavelengths = 2 ** np.arange(0, n-2) * wavelengthMin
#     print('here1')
#     deltaTheta = 30
#     orientations = np.arange(0, 180, deltaTheta)

#     # Create Gabor filters
#     gabors = [gabor_kernel(frequency=1.0/l, theta=np.deg2rad(theta)) for l in wavelengths for theta in orientations]
#     gabormag = np.stack([convolve(image_gray, np.real(g), mode='wrap') for g in gabors], axis=2)
#     print('here2')

#     # Apply Gaussian smoothing
#     for i in range(len(gabors)):
#         sigma = 0.5 * wavelengths[i // len(orientations)]
#         K = 3
#         gabormag[:, :, i] = filters.gaussian(gabormag[:, :, i], sigma=K * sigma)
#     print('here3')

#     # Feature matrix construction
#     X, Y = np.meshgrid(np.arange(numCols), np.arange(numRows))
#     featureSet = np.dstack([gabormag, X, Y])

#     # Reshape and standardize features
#     numPoints = numRows * numCols
#     X = featureSet.reshape((numPoints, -1))
#     scaler = StandardScaler()
#     X = scaler.fit_transform(X)
#     print('here4')

#     # PCA for dimensionality reduction
#     pca = PCA(n_components=1)
#     X_pca = pca.fit_transform(X)
#     feature2DImage = X_pca.reshape((numRows, numCols))
#     plt.figure()
#     plt.imshow(feature2DImage, cmap='jet')
#     plt.axis('on')
#     print('here5')

#     # K-means clustering
#     kmeans = KMeans(n_clusters=2, n_init=5)
#     L = kmeans.fit_predict(X)
#     L = L.reshape((numRows, numCols))
#     print('here6')

#     plt.figure()
#     plt.imshow(color.label2rgb(L, image_gray, kind='avg'))
#     print('here7')

#     # Segment the image
#     Aseg1 = np.zeros_like(image)
#     Aseg2 = np.zeros_like(image)
#     BW = L == 2
#     BW = np.repeat(BW[:, :, np.newaxis], 3, axis=2)
#     Aseg1[BW] = image[BW]
#     Aseg2[~BW] = image[~BW]
#     plt.figure()
#     plt.imshow(np.hstack([Aseg1, Aseg2]))
#     print('here7')

#     print("Elapsed time: {:.2f} seconds".format(time.time() - start_time))
#     plt.show()


In [ ]:
# import os
# import cv2
# import numpy as np
# import xml.etree.ElementTree as ET
# from skimage import io
# from skimage.color import rgb2hed
# from skimage.filters import gaussian, threshold_otsu
# from scipy.ndimage import distance_transform_edt
# from skimage.segmentation import watershed
# from skimage.measure import label
# from skimage.morphology import disk, opening, closing, remove_small_objects, remove_small_holes

# def save_image(image, output_dir, filename):
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
#     if image.dtype != np.uint8:
#         if np.max(image) > 1:
#             image = (255 * (image - np.min(image)) / np.ptp(image)).astype(np.uint8)
#         else:
#             image = (image * 255).astype(np.uint8)
#     io.imsave(os.path.join(output_dir, filename), image)

# def apply_morphological_operations(mask):
#     struct_elem = disk(3)
#     mask_opened = opening(mask, struct_elem)
#     mask_cleaned = closing(mask_opened, struct_elem)
#     mask_cleaned = remove_small_objects(mask_cleaned.astype(bool), min_size=40)
#     mask_cleaned = remove_small_holes(mask_cleaned, area_threshold=40)
#     return (mask_cleaned * 255).astype(np.uint8)

# def parse_xml_boundary_mask(xml_path, image_shape):
#     mask = np.zeros(image_shape[:2], dtype=np.uint8)
#     tree = ET.parse(xml_path)
#     root = tree.getroot()
#     for region in root.findall('.//Region'):
#         vertices = []
#         for vertex in region.findall('.//Vertex'):
#             x = int(float(vertex.get('X')))
#             y = int(float(vertex.get('Y')))
#             vertices.append((x, y))
#         cv2.fillPoly(mask, [np.array(vertices, np.int32)], color=255)
#     return mask

# def process_image(image_path, mask_image_path, output_dir, step_output_dir):
#     base_filename = os.path.splitext(os.path.basename(image_path))[0]
#     image = cv2.imread(image_path)
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     save_image(image_rgb, step_output_dir, f"{base_filename}_01_original.jpg")
    
#     final_mask = cv2.imread(mask_image_path, cv2.IMREAD_GRAYSCALE)
#     save_image(final_mask, step_output_dir, f"{base_filename}_02_final_mask.jpg")

#     hed = rgb2hed(image_rgb)
#     hematoxylin_channel = hed[:, :, 0]

#     blurred = gaussian(hematoxylin_channel, sigma=1)
#     thresh = threshold_otsu(blurred)
#     binary_nuclei = (blurred > thresh).astype(np.uint8) * 255
#     save_image(binary_nuclei, step_output_dir, f"{base_filename}_03_binary_nuclei.jpg")

#     distance = distance_transform_edt(binary_nuclei)
#     save_image(distance, step_output_dir, f"{base_filename}_04_distance.jpg")

#     distance_in_glom = np.where(final_mask, distance, 0)
#     save_image(distance_in_glom, step_output_dir, f"{base_filename}_05_distance_in_glom.jpg")

#     cleaned_distance = apply_morphological_operations(distance_in_glom)
#     save_image(cleaned_distance, step_output_dir, f"{base_filename}_06_cleaned_distance.jpg")

#     markers = label(cleaned_distance > 0)
#     labels = watershed(-cleaned_distance, markers, mask=binary_nuclei)
#     save_image(labels, step_output_dir, f"{base_filename}_07_labels.jpg")

#     labels_in_glom_only = labels * (final_mask > 0)
#     image_overlay = image_rgb.copy()
#     for label_id in np.unique(labels_in_glom_only):
#         if label_id == 0:
#             continue
#         nucleus_mask = labels_in_glom_only == label_id
#         image_overlay[nucleus_mask] = [255, 0, 0]
#     save_image(image_overlay, step_output_dir, f"{base_filename}_08_overlay_nuclei.jpg")

#     io.imsave(os.path.join(output_dir, f"{base_filename}_overlay_nuclei.jpg"), image_overlay)

#     nuclei_count = len(np.unique(labels[final_mask > 0])) - 1
#     print(f"Nuclei count in {base_filename} within glom area: {nuclei_count}")
#     return nuclei_count

# def process_folder(data_folder, masks_folder, output_folder, step_output_folder):
#     nuclei_counts = {}
#     for filename in os.listdir(data_folder):
#         if filename.endswith('.png') and 'patch' in filename:
#             image_path = os.path.join(data_folder, filename)
#             # Construct mask filename based on the expected naming convention
#             mask_image_filename = f"{os.path.splitext(filename)[0].replace('glomerulus', 'glomerulus_mask')}.png"
#             mask_image_path = os.path.join(masks_folder, mask_image_filename)
#             base_filename = os.path.splitext(filename)[0]
#             if os.path.exists(mask_image_path):
#                 nuclei_counts[base_filename] = process_image(image_path, mask_image_path, output_folder, step_output_folder)
#             else:
#                 print(f"Mask image not found for {filename}")

#     counts_filename = os.path.join(output_folder, 'nuclei_counts.txt')
#     with open(counts_filename, 'w') as f:
#         for base_filename, count in nuclei_counts.items():
#             f.write(f"{base_filename}: {count}\n")
#     print(f"Nuclei counts saved to {counts_filename}")

# # Paths to the folders
# data_folder = 'patches_directory'
# # masks_folder = 'data/glomeruli_masks'
# output_folder = 'results/segmentation_results'
# # step_output_folder = 'results/step_results'

# process_folder(data_folder, masks_folder, output_folder, step_output_folder)


In [38]:
# import json
# import numpy as np
# import cv2
# from shapely.geometry import shape, Polygon
# from skimage import io
# import os

# # Base directory for data
# BASE_DIR = '/blue/pinaki.sarder/harishwarreddy.k/Intro_Biomed/Nuclei_Segmentation'

# # Directories containing the GeoJSON and segmented image files
# GEOJSON_DIR = os.path.join(BASE_DIR, 'Gloms_segmented_annotation')
# SEGMENTED_IMAGES_DIR = os.path.join(BASE_DIR, 'segment')
# GROUND_TRUTH_IMAGES_DIR = os.path.join(BASE_DIR, 'gt_img')

# # Create the directory for ground truth images if it does not exist
# if not os.path.exists(GROUND_TRUTH_IMAGES_DIR):
#     os.makedirs(GROUND_TRUTH_IMAGES_DIR)

# # Function to parse the GeoJSON file and return the annotations as polygons
# def parse_geojson(geojson_file):
#     with open(geojson_file, 'r') as file:
#         data = json.load(file)
#     annotations = [shape(feature["geometry"]).buffer(0) for feature in data["features"]]
#     return annotations

# # Function to create binary masks from polygons
# def create_mask_from_polygons(polygons, image_shape):
#     mask = np.zeros(image_shape, dtype=np.uint8)
#     for poly in polygons:
#         if isinstance(poly, Polygon):
#             exterior = np.array(poly.exterior.coords).round().astype(np.int32)
#             cv2.fillPoly(mask, [exterior], 255)
#         else:
#             for geom in poly.geoms:
#                 exterior = np.array(geom.exterior.coords).round().astype(np.int32)
#                 cv2.fillPoly(mask, [exterior], 255)
#     return mask

# # Function to calculate IoU
# def calculate_iou(mask1, mask2):
#     intersection = np.logical_and(mask1, mask2)
#     union = np.logical_or(mask1, mask2)
#     iou_score = np.sum(intersection) / np.sum(union)
#     return iou_score

# # List of GeoJSON files sorted numerically
# geojson_files = sorted(
#     [f for f in os.listdir(GEOJSON_DIR) if f.endswith('.geojson')],
#     key=lambda x: int(x.split('_')[1])
# )

# # Process each GeoJSON file and calculate IoU
# iou_scores = []
# for geojson_file in geojson_files:
#     geojson_path = os.path.join(GEOJSON_DIR, geojson_file)
#     annotations = parse_geojson(geojson_path)

#     # Construct the image file name based on the GeoJSON file name
#     base_name = geojson_file.split('.')[0]
#     image_file_name = f'{base_name}_07_cleaned_distance.jpg'
#     segmented_image_path = os.path.join(SEGMENTED_IMAGES_DIR, image_file_name)

#     # Load the corresponding segmented image
#     segmented_image = io.imread(segmented_image_path, as_gray=True)
#     if len(segmented_image.shape) > 2:
#         segmented_image = segmented_image[:, :, 0]  # Use the first channel if RGB

#     # Ensure it is a binary mask
#     segmented_mask = segmented_image > 127  # Thresholding, adjust value as needed

#     # Image dimensions from the segmented image
#     image_shape = segmented_mask.shape

#     # Create binary masks for all annotations
#     ground_truth_mask = create_mask_from_polygons(annotations, image_shape)

#     # Save the ground truth mask as an image
#     ground_truth_image_path = os.path.join(GROUND_TRUTH_IMAGES_DIR, f"{base_name}_mask.png")
#     io.imsave(ground_truth_image_path, ground_truth_mask.astype(np.uint8))

#     # Calculate IoU for the current image
#     iou_score = calculate_iou(ground_truth_mask, segmented_mask)
#     iou_scores.append(iou_score)

# # Average IoU score
# average_iou = np.mean(iou_scores)
# print(f'Average IoU score for the dataset is: {average_iou}')


# # Average IoU score for the dataset is: 0.6535624494064117

In [36]:
# import os
# import cv2
# import numpy as np
# import xml.etree.ElementTree as ET
# from skimage import io
# from skimage.color import rgb2hed
# from skimage.filters import gaussian, threshold_otsu
# from scipy.ndimage import distance_transform_edt
# from skimage.segmentation import watershed
# from skimage.measure import label
# from skimage.morphology import disk, opening, closing, remove_small_objects, remove_small_holes

# def save_image(image, output_dir, filename):
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
#     if image.dtype != np.uint8:
#         if np.max(image) > 1:
#             image = (255 * (image - np.min(image)) / np.ptp(image)).astype(np.uint8)
#         else:
#             image = (image * 255).astype(np.uint8)
#     io.imsave(os.path.join(output_dir, filename), image)

# def apply_morphological_operations(mask):
#     struct_elem = disk(3)
#     mask_opened = opening(mask, struct_elem)
#     mask_cleaned = closing(mask_opened, struct_elem)
#     mask_cleaned = remove_small_objects(mask_cleaned.astype(bool), min_size=40)
#     mask_cleaned = remove_small_holes(mask_cleaned, area_threshold=40)
#     return (mask_cleaned * 255).astype(np.uint8)

# def parse_xml_boundary_mask(xml_path, image_shape):
#     mask = np.zeros(image_shape[:2], dtype=np.uint8)
#     tree = ET.parse(xml_path)
#     root = tree.getroot()
#     for region in root.findall('.//Region'):
#         vertices = []
#         for vertex in region.findall('.//Vertex'):
#             x = int(float(vertex.get('X')))
#             y = int(float(vertex.get('Y')))
#             vertices.append((x, y))
#         cv2.fillPoly(mask, [np.array(vertices, np.int32)], color=255)
#     return mask

In [ ]:
# def process_image(image_path, mask_image_path, output_dir, step_output_dir):
#     base_filename = os.path.splitext(os.path.basename(image_path))[0]
#     image = cv2.imread(image_path)
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     # save_image(image_rgb, step_output_dir, f"{base_filename}_01_original.jpg")
    
#     final_mask = cv2.imread(mask_image_path, cv2.IMREAD_GRAYSCALE)
#     # save_image(final_mask, step_output_dir, f"{base_filename}_02_final_mask.jpg")

#     hed = rgb2hed(image_rgb)
#     hematoxylin_channel = hed[:, :, 0]

#     blurred = gaussian(hematoxylin_channel, sigma=1)
#     thresh = threshold_otsu(blurred)
#     binary_nuclei = (blurred > thresh).astype(np.uint8) * 255
#     # save_image(binary_nuclei, step_output_dir, f"{base_filename}_03_binary_nuclei.jpg")

#     distance = distance_transform_edt(binary_nuclei)
#     # save_image(distance, step_output_dir, f"{base_filename}_04_distance.jpg")

#     distance_in_glom = np.where(final_mask, distance, 0)
#     # save_image(distance_in_glom, step_output_dir, f"{base_filename}_05_distance_in_glom.jpg")

#     cleaned_distance = apply_morphological_operations(distance_in_glom)
#     save_image(cleaned_distance, step_output_dir, f"{base_filename}_06_cleaned_distance.jpg")

#     markers = label(cleaned_distance > 0)
#     labels = watershed(-cleaned_distance, markers, mask=binary_nuclei)
#     save_image(labels, step_output_dir, f"{base_filename}_07_labels.jpg")

# #     labels_in_glom_only = labels * (final_mask > 0)
# #     image_overlay = image_rgb.copy()
# #     for label_id in np.unique(labels_in_glom_only):
# #         if label_id == 0:
# #             continue
# #         nucleus_mask = labels_in_glom_only == label_id
# #         image_overlay[nucleus_mask] = [255, 0, 0]
# #     save_image(image_overlay, step_output_dir, f"{base_filename}_08_overlay_nuclei.jpg")

# #     io.imsave(os.path.join(output_dir, f"{base_filename}_overlay_nuclei.jpg"), image_overlay)

# #     nuclei_count = len(np.unique(labels[final_mask > 0])) - 1
# #     print(f"Nuclei count in {base_filename} within glom area: {nuclei_count}")
#     return nuclei_count

# # def process_folder(data_folder, masks_folder, output_folder, step_output_folder):
# #     nuclei_counts = {}
# #     for filename in os.listdir(data_folder):
# #         if filename.endswith('.png') and 'glomerulus' in filename:
# #             image_path = os.path.join(data_folder, filename)
# #             # Construct mask filename based on the expected naming convention
# #             mask_image_filename = f"{os.path.splitext(filename)[0].replace('glomerulus', 'glomerulus_mask')}.png"
# #             mask_image_path = os.path.join(masks_folder, mask_image_filename)
# #             base_filename = os.path.splitext(filename)[0]
# #             if os.path.exists(mask_image_path):
# #                 nuclei_counts[base_filename] = process_image(image_path, mask_image_path, output_folder, step_output_folder)
# #             else:
# #                 print(f"Mask image not found for {filename}")

# #     counts_filename = os.path.join(output_folder, 'nuclei_counts.txt')
# #     with open(counts_filename, 'w') as f:
# #         for base_filename, count in nuclei_counts.items():
# #             f.write(f"{base_filename}: {count}\n")
# #     print(f"Nuclei counts saved to {counts_filename}")

# # Paths to the folders
# data_folder = 'Gloms_segmented'
# masks_folder = 'glomeruli_masks'
# output_folder = 'Seg_glom_resutls_nuclei'
# step_output_folder = 'results/step_results'

# process_folder(data_folder, output_folder, step_output_folder)
